In [ ]:
import moldesign as mdt
import moldesign.units as u

In [ ]:
#mdt.configure()

In [ ]:
molecule = mdt.from_pdb('1yu8')

In [ ]:
molecule
#molecule = mdt.Molecule([res for res in molecule.residues if res.type == 'protein'])

In [ ]:
molecule.draw()
#len(molecule.atoms)
u.femtoseconds

In [ ]:
import parmed as med

In [ ]:
mdmol = mdt.assign_forcefield(molecule)

In [ ]:
mdmol.ff.amber_params.prmtop.put('/tmp/tmp.prmtop')
parmedmol = med.load_file('/tmp/tmp.prmtop')

In [ ]:
# mdmol.draw()

In [ ]:

# In order to simluate using LAMMPS, we need:
#   get molecule from pdb, cif, or name
#   assign forcefields
#   load parmed structure


# LAMMPS UNITS:
# For style real, these are the units:

# mass = grams/mole
# distance = Angstroms
# time = femtoseconds
# energy = Kcal/mole
# velocity = Angstroms/femtosecond
# force = Kcal/mole-Angstrom
# torque = Kcal/mole
# temperature = Kelvin
# pressure = atmospheres
# dynamic viscosity = Poise
# charge = multiple of electron charge (1.0 is a proton)
# dipole = charge*Angstroms
# electric field = volts/Angstrom
# density = gram/cm^dim

# See http://parmed.github.io/ParmEd/html/index.html for ParmEd units

def formatLammpsData(dataName):
    filePath = '/tmp/data.' + dataName
    lammps_data = open(filePath, 'w')

    # print description
    lammps_data.write("LAMMPS Description\r\n\r\n")
    lammps_data.write("{0} atoms\r\n" .format(len(parmedmol.atoms)))
    lammps_data.write("{0} bonds\r\n" .format(len(parmedmol.bonds)))
    lammps_data.write("{0} angles\r\n" .format(len(parmedmol.angles)))
    lammps_data.write("{0} dihedrals\r\n" .format(len(parmedmol.dihedrals)))
    lammps_data.write("{0} impropers\r\n" .format(len(parmedmol.impropers)))
    
    lammps_data.write("\r\n")
    
    max_idx = 0
    for nonbond_idx in parmedmol.LJ_types.itervalues():
        if(nonbond_idx > max_idx):
            max_idx = nonbond_idx

    lammps_data.write("{0} atom types\r\n" .format(max_idx))
    lammps_data.write("{0} bond types\r\n" .format(len(parmedmol.bond_types)))
    lammps_data.write("{0} angle types\r\n" .format(len(parmedmol.angle_types)))
    lammps_data.write("{0} dihedral types\r\n" .format(len(parmedmol.dihedral_types)))
    lammps_data.write("{0} improper types\r\n" .format(len(parmedmol.improper_types)))
    
    lammps_data.write("\r\n")

    # box size
    # calculate lo and hi points
    xlo = xhi = ylo = yhi = zlo = zhi = None
    for atom in mdmol.atoms:
        if xlo == None:
            xlo = atom.x.value_in(u.angstrom)
            xhi = atom.x.value_in(u.angstrom)
            ylo = atom.y.value_in(u.angstrom)
            yhi = atom.y.value_in(u.angstrom)
            zlo = atom.z.value_in(u.angstrom)
            zhi = atom.z.value_in(u.angstrom)
        else:
            xlo = min(xlo, atom.x.value_in(u.angstrom))
            xhi = max(xhi, atom.x.value_in(u.angstrom))
            ylo = min(ylo, atom.y.value_in(u.angstrom))
            yhi = max(yhi, atom.y.value_in(u.angstrom))
            zlo = min(zlo, atom.z.value_in(u.angstrom))
            zhi = max(zhi, atom.z.value_in(u.angstrom))

    lammps_data.write("{0} {1} xlo xhi\r\n" .format(xlo, xhi))
    lammps_data.write("{0} {1} ylo yhi\r\n" .format(ylo, yhi))
    lammps_data.write("{0} {1} zlo zhi\r\n" .format(zlo, zhi))
    
    lammps_data.write("\r\n")

    # print masses -- Number of atom types
    lammps_data.write("Masses\r\n\r\n")
    for i in range(1, max_idx+1):
        for atom in parmedmol.atoms:
            if atom.nb_idx == i:
                lammps_data.write("{0} {1}\r\n" .format(i, atom.mass))
                break
    
    lammps_data.write("\r\n")
    

    # Pair Coeffs -- Number of atom types
    lammps_data.write("Pair Coeffs\r\n\r\n")
    for i in range(1, max_idx+1):
        for atom in parmedmol.atoms:
            if atom.nb_idx == i:
                lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(i, atom.epsilon, atom.sigma, 
                                                                    atom.epsilon_14, atom.sigma_14))
                break
    
    lammps_data.write("\r\n")
    

    # Bond Coeffs - Harmonic
    if parmedmol.bond_types:
        lammps_data.write("Bond Coeffs\r\n\r\n")
        for bt in parmedmol.bond_types:
            lammps_data.write("{0} {1} {2}\r\n" .format(bt.idx+1, bt.k, bt.req))
        
        lammps_data.write("\r\n")


    # Angle Coeffs - Harmonic
    if parmedmol.angle_types:
        lammps_data.write("Angle Coeffs\r\n\r\n")
        for at in parmedmol.angle_types:
            lammps_data.write("{0} {1} {2}\r\n" .format(at.idx+1, at.k, at.theteq))
       
        lammps_data.write("\r\n")


    # Dihedral Coeffs - Charmm
    if parmedmol.dihedral_types:
        lammps_data.write("Dihedral Coeffs\r\n\r\n")
        for dt in parmedmol.dihedral_types:
            lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(dt.idx+1, dt.phi_k, dt.per, 
                (180 if dt.phase >= 180 else 0), 
                (1.0 if dt.scnb == 1.0 else 0.0)))
        
        lammps_data.write("\r\n")


    # Improper Coeffs - Harmonic
    if parmedmol.improper_types:
        lammps_data.write("Improper Coeffs\r\n\r\n")
        for it in parmedmol.improper_types:
            print
            lammps_data.write("{0} {1} {2}\r\n" .format(it.idx+1, it.psi_k, it.psi_eq))
        
        lammps_data.write("\r\n")


    # print atoms
    lammps_data.write("Atoms\r\n\r\n")
    for atom in parmedmol.atoms:
        lammps_data.write("{0} {1} {2} {3} " .format(atom.idx+1, atom.residue.idx, atom.nb_idx, atom.charge))
        if not mdmol.positions.any():        
            lammps_data.write("{0} {1} {2} " .format(0.00, 0.00, 0.00))        
        else:
            pos = mdmol.positions[atom.idx]
            lammps_data.write("{0} {1} {2} " .format(pos[0].value_in(u.angstrom), pos[1].value_in(u.angstrom), 
                pos[2].value_in(u.angstrom)))
        lammps_data.write("{0} {1} {2}\r\n" .format(0, 0, 0)) # nx, ny, nz not defined    

    lammps_data.write("\r\n")


    # print velocities
    lammps_data.write("Velocities\r\n\r\n")
    for atom in mdmol.atoms:
        xvel = atom.vx.value_in(u.angstrom/u.fs)
        yvel = atom.vy.value_in(u.angstrom/u.fs)
        zvel = atom.vz.value_in(u.angstrom/u.fs)
        lammps_data.write("{0} {1} {2} {3}\r\n" .format(atom.index+1, xvel, yvel, zvel))
        
    lammps_data.write("\r\n")


    # print bonds
    lammps_data.write("Bonds\r\n\r\n")
    for idx, bond in enumerate(parmedmol.bonds):
        lammps_data.write("{0} {1} {2} {3}\r\n" .format(idx+1, bond.type.idx+1, 
          bond.atom1.idx+1, bond.atom2.idx+1))
        
    lammps_data.write("\r\n")


    # print angles
    lammps_data.write("Angles\r\n\r\n")
    for idx, angle in enumerate(parmedmol.angles):
        lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(idx+1, angle.type.idx+1,
          angle.atom1.idx+1, 
          angle.atom2.idx+1, angle.atom3.idx+1))
        
    lammps_data.write("\r\n")


    # print dihedrals
    if parmedmol.dihedrals:
        lammps_data.write("Dihedrals\r\n\r\n")
        for idx, di in enumerate(parmedmol.dihedrals):
            lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(idx+1, di.type.idx+1, 
              di.atom1.idx+1, di.atom2.idx+1, 
              di.atom3.idx+1, di.atom4.idx+1))
            
        lammps_data.write("\r\n")


    # print impropers
    if parmedmol.impropers:
        lammps_data.write("Impropers\r\n\r\n")
        for im in parmedmol.impropers:
            lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(im.idx+1, im.type.idx+1, 
              im.atom1.idx+1, im.atom2.idx+1, 
              im.atom3.idx+1, im.atom4.idx+1))
        
        lammps_data.write("\r\n")

    
    lammps_data.close()
    return filePath

In [ ]:
name = formatLammpsData('test')

In [ ]:
def prepareLammps(dataPath):
    L.command("units real")
    L.command("atom_style full")
    L.command("pair_style lj/charmm/coul/long 8.0 10.0 10.0")
    L.command("bond_style harmonic")
    L.command("angle_style harmonic")
    L.command("dihedral_style charmm")
    L.command("improper_style harmonic")
    L.command("kspace_style pppm 0.0001")
    L.command("read_data " + dataPath)

def calculateEnergy(timeStep, outputFreq):
    #L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    #L.command("thermo " + str(outputFreq))

            
def groupHydrogens():
    groupCommand = "group hbonds type"
    for nonbond_name, nonbond_idx in parmedmol.LJ_types.iteritems():
        if(nonbond_name[0] == 'H'):
            groupCommand = groupCommand + " " + str(nonbond_idx)
    L.command(groupCommand)
    
    
def runNvt(timeStep, temp, damp, outputFreq, applyShake):
    L.command("neighbor 2.0 bin")
    #L.command("neigh_modify delay 5")
    L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    L.command("thermo " + str(outputFreq))
    nvt_command = "fix 1 all nvt temp {0} {1} {2}" .format(round(temp,1), round(temp,1), round(damp, 3))
    L.command(nvt_command)
    if(applyShake):
        groupHydrogens()
        shake_command = "fix 2 hbonds rattle 0.0001 20 0 t 5 6 m 1.0 a 31"
        L.command(shake_command)


In [ ]:
from lammps import lammps, PyLammps

In [ ]:
lmp = lammps()
L = PyLammps(ptr=lmp)

In [ ]:
prepareLammps(name)

In [ ]:
print "Before Evaluation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
calculateEnergy(1.0, 10)
L.run(1000)
print L.eval("pe")


In [ ]:
print L.atoms.natoms
#for i in range(0, L.atoms.natoms):
    #print L.atoms[i].force

In [ ]:
print "After Evaluation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
del L
del lmp

lmp = lammps()
L = PyLammps(ptr=lmp)

In [ ]:
prepareLammps(name)

In [ ]:
print "Before Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
runNvt(1.0, 300, 100, 10, True)


In [ ]:
# shake_command = "fix 2 hbonds shake 0.0001 10 0 b 4 6 8 10 12 14 18 a 31"
# L.command(shake_command)


In [ ]:
L.run(200)

In [ ]:
print "After Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
#

In [ ]:
# mdmol.draw3d() 17

In [ ]:
for j in range(0, 10):
    L.run(200)
    print "After Simulation"
    print L.atoms[1012].position
    print L.atoms[1012].velocity
    print L.atoms[1012].force
    

In [ ]:
L.run(10)

In [ ]:
for j in range(0, 10):
    L.run(1)
    print "After Simulation"
    print L.atoms[1012].position
    print L.atoms[1012].velocity
    print L.atoms[1012].force

In [ ]:
print L.atoms.natoms
i = 0
for atom in mdmol.velocities:
    i+=1
print i

